In [80]:
import sys

sys.path.append('../_util')
import pandas as pd
import numpy as np

data = pd.read_csv('../data/Data_Marketing_Customer_Analysis_Round3.csv')
from _util import categorical_numerical_spliter
# from _util import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler


In [81]:
def df_cleaner(df):
    df = df.drop(
        ['effective_to_date', 'response', 'total_claim_amount', 'customer_lifetime_value'],
        errors='ignore',
        axis='columns'
    )
    df = df.reindex(sorted(df.columns), axis=1)

    return df

In [82]:


numerical, categorical = categorical_numerical_spliter(data)

categorical


,region,response,coverage,education,effective_to_date,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,arizona,no,basic,college,2/18/11,employed,m,suburban,married,corporate auto,corporate l3,offer3,agent,four-door car,medsize
1,california,no,basic,college,1/18/11,unemployed,f,suburban,single,personal auto,personal l3,offer4,call center,four-door car,medsize
2,washington,no,basic,bachelor,2/10/11,employed,m,suburban,single,personal auto,personal l3,offer3,call center,suv,medsize
3,oregon,yes,extended,college,1/11/11,employed,m,suburban,single,corporate auto,corporate l3,offer2,branch,four-door car,medsize
4,oregon,no,premium,bachelor,1/17/11,medical leave,f,suburban,married,personal auto,personal l2,offer1,branch,four-door car,medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10597,nevada,no,premium,bachelor,1/19/11,unemployed,f,suburban,married,personal auto,personal l1,offer3,web,luxury car,medsize
10598,oregon,no,basic,college,1/6/11,employed,f,urban,married,personal auto,personal l3,offer2,branch,four-door car,medsize
10599,arizona,no,extended,bachelor,2/6/11,employed,f,rural,married,corporate auto,corporate l3,offer1,web,luxury suv,medsize
10600,california,no,premium,college,2/13/11,employed,f,urban,divorced,personal auto,personal l1,offer1,branch,suv,medsize


In [83]:
categorical_dummies = pd.get_dummies(categorical, drop_first=True)
categorical_dummies.reset_index(drop=True, inplace=True)
X = pd.concat([numerical, categorical_dummies], axis=1, ignore_index=False)
X = df_cleaner(X)
y = data['total_claim_amount']

,coverage_extended,coverage_premium,education_college,education_doctor,education_high school or below,education_master,effective_to_date_1/10/11,effective_to_date_1/11/11,effective_to_date_1/12/11,effective_to_date_1/13/11,...,sales_channel_branch,sales_channel_call center,sales_channel_web,vehicle_class_luxury car,vehicle_class_luxury suv,vehicle_class_sports car,vehicle_class_suv,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,1,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10597,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
10598,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
10599,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
10600,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0


## Embedded Methods

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train = X_train.select_dtypes(include=np.number)
X_test = X_test.select_dtypes(include=np.number)
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp_mean.fit_transform(X_train)


In [85]:
model = LinearRegression()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

LinearRegression: Train -> 0.7737930198557191, Test -> 0.7656745202259884


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [86]:

model = Lasso(alpha=0.05)

model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

Lasso: Train -> 0.7736738997581394, Test -> 0.7660869660860785


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


In [87]:
model = Ridge(alpha=10000)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

Ridge: Train -> 0.5394160980208348, Test -> 0.5219297169821564


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(
